# Desafio Stone - Distribuição de lucros

## Descrição
Uma empresa fechou o ano de operação com lucro e deseja reparti-lo entre seus funcionários, com o objetivo de ser justa criou uma regra para a distribuição deste montante por: área, tempo de empresa, e faixa salarial (os funcionários que ganham menos teriam sua participação incrementada). Para isso foi solicitado ao time de tecnologia que desenvolva um processo que receba um valor máximo em dinheiro e, olhando a base de dados da empresa, distribua o montante para os funcionários já cadastrados com os dados abaixo.

## Regras Gerais

São 3 pesos. Por area de atuação, por faixa salarial e tempode admissão.

In [285]:
# importando as bibliotecas
import pandas as pd
import datetime 

In [286]:
# criando um dataset a partir da base de dados
path = "/content/base_desafio_lucros.json"
df = pd.read_json(path)

In [287]:
total_disponibilizado = 5000000

In [288]:
# visualizando os primeiros registros
df.head()

,matricula,nome,area,cargo,salario_bruto,data_de_admissao
0,5433534,Ivo Ferreira,Tecnologia,Pessoa Desenvolvedora,"$3,636.00",2004-05-28
1,5241166,Caetana da Magalhães,Tecnologia,Jovem Aprendiz,"$12,726.00",2009-09-06
2,3027629,Benjamim Cardoso,Contabilidade,Auxiliar Administrativo,"$3,636.00",2020-05-08
3,2556888,Diego da Campos,Relacionamento com o Cliente,Atendente,"$1,515.00",2016-09-10
4,2390033,Leonor Moura,Tecnologia,Jovem Aprendiz,"$2,424.00",2016-01-15


In [320]:
# inicialmente vamos criar um df de apoio
df.apoio_pesos = pd.DataFrame()
df.apoio_pesos['matricula'] = df['matricula']

## Peso para area de atuação (mais peso, maior o valor recebido)
* Peso 1: Diretoria;
* Peso 2: Contabilidade, Financeiro, Tecnologia;
* Peso 3: Serviços Gerais;
* Peso 5: Relacionamento com o Cliente;


In [323]:
## trouxe a area provisoriamente para o df para facilitar a conferência visual
df.apoio_pesos['area'] = df['area']
df.apoio_pesos.head()

,matricula,area
0,5433534,Tecnologia
1,5241166,Tecnologia
2,3027629,Contabilidade
3,2556888,Relacionamento com o Cliente
4,2390033,Tecnologia


In [324]:
## fase de tratamento. Inclusão do peso da area
df.apoio_pesos.loc[df.apoio_pesos['area'] == 'Diretoria', 'f1'] = 1
df.apoio_pesos.loc[df.apoio_pesos['area'] == 'Contabilidade', 'f1'] = 2
df.apoio_pesos.loc[df.apoio_pesos['area'] == 'Financeiro', 'f1'] = 2
df.apoio_pesos.loc[df.apoio_pesos['area'] == 'Tecnologia', 'f1'] = 2
df.apoio_pesos.loc[df.apoio_pesos['area'] == 'Serviços Gerais', 'f1'] = 3
df.apoio_pesos.loc[df.apoio_pesos['area'] == 'Relacionamento com o Cliente', 'f1'] = 5

In [325]:
df.apoio_pesos.head()

,matricula,area,f1
0,5433534,Tecnologia,2.0
1,5241166,Tecnologia,2.0
2,3027629,Contabilidade,2.0
3,2556888,Relacionamento com o Cliente,5.0
4,2390033,Tecnologia,2.0


## Segundo peso por faixa salarial (Quanto maior o peso, menos o valor recebido)

Aqui apliquei uma ideia de peso negativo. Defini como regra o peso 6 e reduzi do peso informado no banco de dados. Desta forma, os maiores pesos resultam em pesos menores ao final do processo.

* Peso 5: Acima de 8 salários mínimos; (Final **6** - 5 = Peso 1)
* Peso 3: Acima de 5 salários mínimos e menor que 8 salários mínimos; (Final **6** - 3 = Peso 3)
* Peso 2: Acima de 3 salários mínimos e menor que 5 salários mínimos; (Final **6** - 2 = Peso 3)
* Peso 1: Todos os estagiários e funcionários que ganham até 3 salários mínimos; (Final **6** - 1 = Peso 5)

In [326]:
## trazendo o salário provisoriamente para facilitar a conferência
df.apoio_pesos['salario_bruto'] = df['salario_bruto']
## tratamento de dados
## 1> remoção do cifrão
## 2> remoção da vírgula
## 3> substituição da casa decimal de virgula para ponto
## 4> conversão de object str para float
df.apoio_pesos['salario_formatado'] = df.apoio_pesos['salario_bruto'] .apply(lambda x: str(x.replace("$","")))
df.apoio_pesos['salario_formatado'] = df.apoio_pesos['salario_formatado'].apply(lambda x: str(x).replace(",","") )
df.apoio_pesos['salario_formatado'] = df.apoio_pesos['salario_formatado'].astype('float64')
df.apoio_pesos.drop('salario_bruto', axis=1, inplace=True)
df.apoio_pesos.head()

,matricula,area,f1,salario_formatado
0,5433534,Tecnologia,2.0,3636.0
1,5241166,Tecnologia,2.0,12726.0
2,3027629,Contabilidade,2.0,3636.0
3,2556888,Relacionamento com o Cliente,5.0,1515.0
4,2390033,Tecnologia,2.0,2424.0


In [327]:
## regra para aplicar os pesos
## relembrando a fórmula: 6 - peso da tabela
## primeiro vamos atribuir o valor do salário mínimo
salario_minino = 1212

In [328]:
#todos os estagiários e funcionários que ganham até 3 salários mínimos
df.apoio_pesos.loc[df.apoio_pesos['salario_formatado'] <= (3 * salario_minino), 'f2'] = (6 - 1)
df.apoio_pesos.loc[df.apoio_pesos['f2'] ==5, 'faixa_salarial'] = 1
#peso 3 maior que 3 salários e menor que 5 salários
df.apoio_pesos.loc[df.apoio_pesos['salario_formatado'] > (3 * salario_minino), 'f2'] = (6 - 2)
df.apoio_pesos.loc[df.apoio_pesos['f2'] ==4, 'faixa_salarial'] = 2
#peso 3 maior que 5 salários e menor que 8 salários
df.apoio_pesos.loc[df.apoio_pesos['salario_formatado'] > (5 * salario_minino), 'f2'] = (6 - 3)
df.apoio_pesos.loc[df.apoio_pesos['f2'] ==3, 'faixa_salarial'] = 3
# peso 5 acima de 8 salários mínimos
df.apoio_pesos.loc[df.apoio_pesos['salario_formatado'] > (8 * salario_minino), 'f2'] = (6 - 5)
df.apoio_pesos.loc[df.apoio_pesos['f2'] ==1, 'faixa_salarial'] = 5

In [330]:
# visão parcial de nosso df de apoio
print(df.apoio_pesos.head())

print()
print(f'faixa salarial 1 equivale a f2 = 5> até 3 salários mínimos que corresponde a {salario_minino * 3}')
print(f'faixa salarial 2 equivale a f2 = 4> acima de 3 salários e menor que 5 salários que corresponde até {salario_minino * 5}')
print(f'faixa salarial 3 equivale a f2 = 3> acima de 5 salários e menor que 8 salários que corresponde até {salario_minino * 8}')
print(f'faixa salarial 5 equivale a f2 = 1> acima de 8 salários que corresponde a valores acima de {salario_minino * 8}')



   matricula                          area   f1  salario_formatado   f2  \
0    5433534                    Tecnologia  2.0             3636.0  5.0   
1    5241166                    Tecnologia  2.0            12726.0  1.0   
2    3027629                 Contabilidade  2.0             3636.0  5.0   
3    2556888  Relacionamento com o Cliente  5.0             1515.0  5.0   
4    2390033                    Tecnologia  2.0             2424.0  5.0   

   faixa_salarial  
0             1.0  
1             5.0  
2             1.0  
3             1.0  
4             1.0  

faixa salarial 1 equivale a f2 = 5> até 3 salários mínimos que corresponde a 3636
faixa salarial 2 equivale a f2 = 4> acima de 3 salários e menor que 5 salários que corresponde até 6060
faixa salarial 3 equivale a f2 = 3> acima de 5 salários e menor que 8 salários que corresponde até 9696
faixa salarial 5 equivale a f2 = 1> acima de 8 salários que corresponde a valores acima de 9696


# Terceiro peso com cálculo do tempo de admissão
* Peso 1: Até 1 ano de casa;
* Peso 2: Mais de 1 ano e menos de 3 anos;
* Peso 3: Acima de 3 anos e menos de 8 anos;
* Peso 5: Mais de 8 anos

In [331]:
## gerando a data do dia
hoje = datetime.date.today()
## trazendo o salário provisoriamente para facilitar a conferência
df.apoio_pesos['data_de_admissao'] = df['data_de_admissao']
## convertendo object para data
df.apoio_pesos['data_de_admissao'] = pd.to_datetime(df.apoio_pesos['data_de_admissao']).dt.date
## calculo dos dias
df.apoio_pesos['dias'] = hoje - df.apoio_pesos['data_de_admissao']
## o resultado gera um object time delta. Convertendo em string para posterior conversão
df.apoio_pesos['dias'] = df.apoio_pesos['dias'].astype('string')
## aplicando o replace para remover o "days" e posterior conversão em inteiro
df.apoio_pesos['dias'] = df.apoio_pesos['dias'].apply(lambda x: str(x.replace(" days","")))
## convertendo para inteiro
df.apoio_pesos['dias'] = df.apoio_pesos['dias'].astype('int64')
df.apoio_pesos.head()


,matricula,area,f1,salario_formatado,f2,faixa_salarial,data_de_admissao,dias
0,5433534,Tecnologia,2.0,3636.0,5.0,1.0,2004-05-28,6569
1,5241166,Tecnologia,2.0,12726.0,1.0,5.0,2009-09-06,4642
2,3027629,Contabilidade,2.0,3636.0,5.0,1.0,2020-05-08,745
3,2556888,Relacionamento com o Cliente,5.0,1515.0,5.0,1.0,2016-09-10,2081
4,2390033,Tecnologia,2.0,2424.0,5.0,1.0,2016-01-15,2320


In [332]:
# calculo dos anos contando o número de dias para fechar os anos
# resultado armazenado na coluna f3
# f3 corresponde ao peso
df.apoio_pesos.loc[df.apoio_pesos['dias'] > 2920, 'f3'] = 5
df.apoio_pesos.loc[df.apoio_pesos['dias'] <= 2920, 'f3'] = 3
df.apoio_pesos.loc[df.apoio_pesos['dias'] <= 1095, 'f3'] = 2
df.apoio_pesos.loc[df.apoio_pesos['dias'] <= 365, 'f3'] = 1

df.apoio_pesos.head()


,matricula,area,f1,salario_formatado,f2,faixa_salarial,data_de_admissao,dias,f3
0,5433534,Tecnologia,2.0,3636.0,5.0,1.0,2004-05-28,6569,5.0
1,5241166,Tecnologia,2.0,12726.0,1.0,5.0,2009-09-06,4642,5.0
2,3027629,Contabilidade,2.0,3636.0,5.0,1.0,2020-05-08,745,2.0
3,2556888,Relacionamento com o Cliente,5.0,1515.0,5.0,1.0,2016-09-10,2081,3.0
4,2390033,Tecnologia,2.0,2424.0,5.0,1.0,2016-01-15,2320,3.0


## Gerando valores para totalizar os pesos
Os totais estão sendo calculados e replicados na tabela. Assim a fórmula é executada a nível de linha para apurar  o valor do bônus de cada funcionário

In [333]:
# aplica sum na série e salva na variável
soma_f1  = df.apoio_pesos['f1'].sum()
soma_f2  = df.apoio_pesos['f2'].sum()
soma_f3  = df.apoio_pesos['f3'].sum()
# atribui o soma dos valores a variável soma_total_fatores
soma_total_fatores = soma_f1 + soma_f2 + soma_f3
print(soma_f1)
print(soma_f2)
print(soma_f3)
print(soma_total_fatores)

2389.0
3321.0
4050.0
9760.0


In [334]:
## adicionando ao data frame de apoio os cálculos gerados acima
df.apoio_pesos['soma_f1'] = soma_f1
df.apoio_pesos['soma_f2'] = soma_f2
df.apoio_pesos['soma_f3'] = soma_f3
df.apoio_pesos['soma_total_fatores'] = soma_total_fatores

# calculo do peso individual de cada funcionário
df.apoio_pesos['soma_fator_pessoal'] = df.apoio_pesos['f1'] + df.apoio_pesos['f2'] + df.apoio_pesos['f3'] 

# gera o fator do funcionário. É a proporção individual em relação a soma total de fatores
df.apoio_pesos['fator_calculado'] = df.apoio_pesos['soma_fator_pessoal'] / df.apoio_pesos['soma_total_fatores']

# gera o cálculo do bonus. Basicamente multiplicamos o valor pelo valor disponibilizado em bonus
df.apoio_pesos['bonus_a_receber'] = total_disponibilizado * df.apoio_pesos['fator_calculado']

In [336]:
# para fins de controle e gerência, salvando o valor do bonus no df principal
df['bonus_a_receber'] = df.apoio_pesos['bonus_a_receber']
df.head()

,matricula,nome,area,cargo,salario_bruto,data_de_admissao,bonus_a_receber
0,5433534,Ivo Ferreira,Tecnologia,Pessoa Desenvolvedora,"$3,636.00",2004-05-28,6147.540984
1,5241166,Caetana da Magalhães,Tecnologia,Jovem Aprendiz,"$12,726.00",2009-09-06,4098.360656
2,3027629,Benjamim Cardoso,Contabilidade,Auxiliar Administrativo,"$3,636.00",2020-05-08,4610.655738
3,2556888,Diego da Campos,Relacionamento com o Cliente,Atendente,"$1,515.00",2016-09-10,6659.836066
4,2390033,Leonor Moura,Tecnologia,Jovem Aprendiz,"$2,424.00",2016-01-15,5122.950820


# Calculando as informações gerenciais

## Total distribuído = Soma do que foi pago a todos os funcionários

In [312]:
total_distribuido = round(df.apoio_pesos['bonus_a_receber'].sum(),2)
print(f'O valor total distribuído foi de R$ {total_distribuido}')

O valor total distribuído foi de R$ 5000000.0


## Total disponibilizado = O valor que a empresa deseja distribuir

In [313]:
print(f'O valor total disponibilizado informado foi de R$ {total_disponibilizado}')

O valor total disponibilizado informado foi de R$ 5000000


## Total distribuído e número de funcionários por área

In [314]:
total_por_area = df.apoio_pesos.groupby('area').agg('sum')['bonus_a_receber']
qtd_funcionarios_area = df.apoio_pesos.groupby('area').agg('count')['matricula']

print(f'Numero de funcionários por \n {qtd_funcionarios_area}')
print()
print(f'Total distribuído por \n  {round(total_por_area,2)}')

Numero de funcionários por 
 area
Contabilidade                   155
Diretoria                       206
Financeiro                      159
Relacionamento com o Cliente    142
Serviços Gerais                 169
Tecnologia                      169
Name: matricula, dtype: int64

Total distribuído por 
  area
Contabilidade                   720799.18
Diretoria                       726946.72
Financeiro                      769979.51
Relacionamento com o Cliente    999487.70
Serviços Gerais                 955942.62
Tecnologia                      826844.26
Name: bonus_a_receber, dtype: float64


## Total distribuído e número de funcionários por faixa salarial

In [315]:
total_por_faixa_salarial = round(df.apoio_pesos.groupby('faixa_salarial').agg('sum')['bonus_a_receber'],2)
print(f'O valor distribuído por \n {total_por_faixa_salarial} ')
print()
qtd_funcionarios_faixa_salarial = df.apoio_pesos.groupby('faixa_salarial').agg('count')['matricula']
print(f'Número de funcionários por \n {qtd_funcionarios_faixa_salarial} ')

O valor distribuído por 
 faixa_salarial
1.0    1906250.00
2.0    1154200.82
3.0    1102971.31
5.0     836577.87
Name: bonus_a_receber, dtype: float64 

Número de funcionários por 
 faixa_salarial
1.0    308
2.0    205
3.0    237
5.0    250
Name: matricula, dtype: int64 


## Total distribuído e número de funcionários por tempo de admissão

In [318]:
total_por_tempo_admissao = round(df.apoio_pesos.groupby('f3').agg('sum')['bonus_a_receber'],2)
print(f'O valor distribuído por \n {total_por_tempo_admissao} ')
print()
qtd_funcionarios_por_tempo_admissao = df.apoio_pesos.groupby('f3').agg('count')['matricula']
print(f'Número de funcionários por \n {qtd_funcionarios_por_tempo_admissao} ')

#df.apoio_pesos['data_de_admissao'].sort_values().tail(10)
print()
print(f'O tempo de admissão (f3) 1.0 são os funcionários com até 1 ano de casa')
print(f'O tempo de admissão (f3) 2.0 são os funcionários com mais de 1 ano e menos de 3 anos')
print(f'O tempo de admissão (f3) 3.0 são os funcionários com mais de 3 anos e menos de 8 anos')
print(f'O tempo de admissão (f3) 5.0 são os funcionários com mais de 8 anos de casa')


O valor distribuído por 
 f3
1.0     123975.41
2.0     446209.02
3.0    1039446.72
5.0    3390368.85
Name: bonus_a_receber, dtype: float64 

Número de funcionários por 
 f3
1.0     38
2.0    112
3.0    231
5.0    619
Name: matricula, dtype: int64 

O tempo de admissão (f3) 1.0 são os funcionários com até 1 ano de casa
O tempo de admissão (f3) 2.0 são os funcionários com mais de 1 ano e menos de 3 anos
O tempo de admissão (f3) 3.0 são os funcionários com mais de 3 anos e menos de 8 anos
O tempo de admissão (f3) 5.0 são os funcionários com mais de 8 anos de casa
